# Run Agent

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate

# Bước 1: Load biến môi trường
load_dotenv()

# Bước 2: Khởi tạo model Gemini 2.0 Flash
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.7
)


In [10]:
# Bước 3: Định nghĩa tool tạo tài liệu Markdown
@tool
def generate_markdown_doc(code: str) -> str:
    """Tạo tài liệu Markdown mô tả nội dung của một đoạn code, với mô tả linh hoạt về chức năng các hàm."""
    try:
        # Trích xuất các hàm từ code
        lines = code.split("\n")
        functions = []
        current_func = []
        in_function = False
        
        for line in lines:
            if line.strip().startswith("def "):
                in_function = True
                current_func = [line]
            elif in_function and line.strip():
                current_func.append(line)
            elif in_function and not line.strip():
                in_function = False
                if current_func:
                    functions.append("\n".join(current_func))
        
        if current_func and in_function:
            functions.append("\n".join(current_func))
        
        # Tạo nội dung Markdown
        markdown = "# Tài liệu cho đoạn code\n\n"
        markdown += "## Mô tả tổng quan\n"
        markdown += "Đoạn code này chứa các hàm Python. Dưới đây là mô tả chi tiết về chức năng của chúng.\n\n"
        
        if functions:
            markdown += "## Các hàm chính\n"
            for func_code in functions:
                # Trích xuất tên hàm
                func_name = func_code.split("def ")[1].split("(")[0].strip()
                
                # Tạo prompt để Gemini suy luận mô tả hàm
                prompt = f"Phân tích đoạn code sau và cung cấp mô tả chi tiết step-by-step, tự nhiên về chức năng của hàm (bằng tiếng Việt):\n```python\n{func_code}\n```"
                response = llm.invoke(prompt)
                description = response.content.strip()
                
                markdown += f"- `{func_name}`: {description}\n"
        else:
            markdown += "## Nội dung\n"
            markdown += "Code không chứa hàm rõ ràng, có thể là logic xử lý chung.\n"
        
        markdown += "\n## Ví dụ sử dụng\n"
        markdown += f"```python\n# Chạy code như sau:\n{code}\n```"
        
        return markdown
    except Exception as e:
        return f"Lỗi khi tạo tài liệu Markdown: {str(e)}"


In [15]:
@tool
def save_markdown_doc(markdown: str, filename: str) -> str:
    """Lưu tài liệu Markdown vào file với tên được chỉ định."""
    if not filename.endswith(".md"):
        filename += ".md"
    if not filename:
        return "Tên file không được để trống."
    if os.path.exists(filename):
        return f"Tập tin {filename} đã tồn tại. Vui lòng chọn tên khác."
    if not os.access(os.path.dirname(filename) or '.', os.W_OK):
        return f"Không có quyền ghi vào thư mục {os.path.dirname(filename) or '.'}."
    if not markdown.strip():
        return "Nội dung tài liệu không được để trống."
    try:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(markdown)
        return f"Tài liệu đã được lưu vào {filename}"
    except Exception as e:
        return f"Lỗi khi lưu tài liệu: {str(e)}"
# Bước 4: Tạo agent với các tool

In [ ]:

# Bước 4: Danh sách tool
tools = [generate_markdown_doc, save_markdown_doc]

# Bước 5: Tạo prompt cho agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là trợ lý thông minh, sử dụng công cụ để tạo tài liệu Markdown cho code với mô tả linh hoạt và chi tiết theo từng bước suy luận.Sau khi đã tạo tài liệu, bạn sẽ lưu nó vào file với tên document_agent.md"), 
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Bước 6: Tạo agent và executor
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Bước 7: Chạy thử
with open("../tool/writing_document_markdown.py")as f:
    sample_code = f.read()
f.close()
response = agent_executor.invoke({"input": f"Tạo tài liệu Markdown cho đoạn code sau:\n{sample_code}"})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `generate_markdown_doc` with `{'code': 'from langchain.tools import tool\nimport os\n\n\n# Tool 1: List contents of the current directory\n@tool\ndef list_directory_contents() -> str:\n    """List all files and directories in the current working directory.\n    \n    Returns:\n        str: A string containing the list of files and directories, or an error message.\n    """\n    try:\n        cwd = os.getcwd()\n        items = os.listdir(cwd)\n        if not items:\n            return "The current directory is empty."\n        return "\\n".join(items)\n    except Exception as e:\n        return f"Error accessing directory: {str(e)}"\n\n# Tool 2: Write content to a Markdown file (from previous response)\n@tool\ndef write_markdown_file(content: str, filename: str) -> str:\n    """Write content to a Markdown file in the current working directory.\n    \n    Args:\n        content (str): The content to write to the Markdown file.\n        f

In [14]:
response['output'] 

'Dưới đây là tài liệu Markdown cho đoạn code bạn đã cung cấp:\n\n```markdown\n# Tài liệu cho đoạn code\n\n## Mô tả tổng quan\nĐoạn code này định nghĩa hai công cụ (tools) sử dụng thư viện `langchain`. Công cụ đầu tiên liệt kê nội dung của thư mục hiện tại, và công cụ thứ hai ghi nội dung vào một file Markdown.\n\n## Các hàm chính\n- `list_directory_contents`: Liệt kê các file và thư mục trong thư mục làm việc hiện tại.\n- `write_markdown_file`: Ghi nội dung vào một file Markdown trong thư mục làm việc hiện tại.\n\n## Chi tiết từng hàm\n\n### `list_directory_contents()`\n\n```python\n@tool\ndef list_directory_contents() -> str:\n    """List all files and directories in the current working directory.\n    \n    Returns:\n        str: A string containing the list of files and directories, or an error message.\n    """\n    try:\n        cwd = os.getcwd()\n        items = os.listdir(cwd)\n        if not items:\n            return "The current directory is empty."\n        return "\\n".join